In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv('ShubmanGill-IPL.csv')

In [3]:
dataset.drop(columns=['Unnamed: 0'],axis=1,inplace=True)

In [4]:
dataset

,p_match,inns,bat,p_bat,team_bat,bowl,p_bowl,team_bowl,ball,ball_id,...,bat_out,wagonX,wagonY,wagonZone,line,length,shot,control,predscore,wprob
0,1178422,1,Shubman Gill,1070173,Kolkata Knight Riders,Lasith Malinga,49758,Mumbai Indians,1,2.01,...,True,202,219,4,OUTSIDE_OFFSTUMP,FULL,PUSH,1.0,165,46.47
1,1178422,1,Shubman Gill,1070173,Kolkata Knight Riders,Lasith Malinga,49758,Mumbai Indians,4,2.04,...,True,154,224,5,OUTSIDE_OFFSTUMP,FULL,ON_DRIVE,1.0,162,43.17
2,1178422,1,Shubman Gill,1070173,Kolkata Knight Riders,Lasith Malinga,49758,Mumbai Indians,5,2.05,...,True,221,193,3,ON_THE_STUMPS,GOOD_LENGTH,FLICK,1.0,160,41.01
3,1178422,1,Shubman Gill,1070173,Kolkata Knight Riders,Lasith Malinga,49758,Mumbai Indians,6,2.06,...,True,136,209,5,OUTSIDE_OFFSTUMP,FULL,COVER_DRIVE,1.0,159,39.94
4,1178422,1,Shubman Gill,1070173,Kolkata Knight Riders,Lasith Malinga,49758,Mumbai Indians,1,5.01,...,True,324,135,2,ON_THE_STUMPS,GOOD_LENGTH,FLICK,1.0,166,47.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2432,1426297,1,Shubman Gill,1070173,Gujarat Titans,Daryl Mitchell,381743,Chennai Super Kings,2,13.02,...,True,325,49,2,WIDE_OUTSIDE_OFFSTUMP,SHORT,HOOK,1.0,255,96.93
2433,1426297,1,Shubman Gill,1070173,Gujarat Titans,Daryl Mitchell,381743,Chennai Super Kings,3,13.03,...,True,137,357,5,OUTSIDE_OFFSTUMP,FULL,ON_DRIVE,0.0,259,97.41
2434,1426297,1,Shubman Gill,1070173,Gujarat Titans,Daryl Mitchell,381743,Chennai Super Kings,4,13.04,...,True,174,205,5,ON_THE_STUMPS,YORKER,DEFENDED,1.0,257,97.18
2435,1426297,1,Shubman Gill,1070173,Gujarat Titans,Daryl Mitchell,381743,Chennai Super Kings,5,13.05,...,True,23,302,6,OUTSIDE_OFFSTUMP,FULL,COVER_DRIVE,1.0,260,97.52


In [5]:
dataset.drop_duplicates(inplace=True)

In [6]:
dataset.groupby(['bat']).sum()

C:\Users\Vijay\AppData\Local\Temp\ipykernel_16880\1964033360.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dataset.groupby(['bat']).sum()


,p_match,inns,p_bat,p_bowl,ball,ball_id,score,out,p_out,over,...,batruns,ballfaced,bowlruns,bat_out,wagonX,wagonY,wagonZone,control,predscore,wprob
bat,,,,,,,,,,,,,,,,,,,,,
Andre Russell,1136586,2,276298,31107,3,15.03,0,1,1070173,16,...,0,1,0,0,123,138,7,1.0,181,0.21
Eoin Morgan,1216540,2,24598,719715,3,9.03,0,1,1070173,10,...,0,1,0,0,240,167,3,0.0,154,3.29
Shubman Gill,3132748499,3669,2602660736,1463825484,8429,16824.29,3327,86,2601436225,19172,...,3216,2370,3298,2429,424761,461639,9943,2026.0,420977,140030.63


In [7]:
reqcols=['inns','bat','team_bat','bowl','team_bowl','ball','ball_id','out','dismissal','over','noball','wide','date','year','ground','country','bat_hand','bowl_style','bowl_kind','batruns','ballfaced','line','length','shot','control']